1987 LOC bibliography
Start of main text: 5746
End of main text: 304122
Start of serials: 304474
End of serials: 322556
End has indices of subject, title, corporate bodies

In [10]:
import re

In [11]:
loc1987obj = open('1987LOC.txt')#, newline='\n')
loc1987 = loc1987obj.read()#lines()

In [12]:
loc87monos_water = loc1987[5746:304122]
loc87serials_water = loc1987[304474:322556]

In [79]:
watermarkpatt = r'Digitized.*?\x0c'
#finds text from word "Digitized" to weird character that shows up in Google bookmarks
watermark = re.compile(watermarkpatt, re.DOTALL)

#find number with newlines -- most obvious divider for records
numberstring = r'\n[0-9]+\n'
numberdiv = re.compile(numberstring)

#find to first period -- often author?
startstring = r'^.*?\.'
start = re.compile(startstring)

lccnstring = r'(?<=\n\n)[0-9]{2}-[ \t]?[0-9]+'#(?=\n\n)'
lccnpat = re.compile(lccnstring)

lcclass = r'(?<=\n\n)[A-Z]+[0-9+].*?(?=\n\n)'
lcclass = re.compile(lcclass, re.DOTALL)

#current ([s.n.], [s.l.] handled separately) 
pubstring= r'(?<=\.)[\w\n\[\], -]*?([0-9]{4}|[0-9]{3,4} ?[?+])'
#below: failed attempt to incorporate [s.n.], [s.l.] -- abandoned
#pubstring= r'(?<=\.) (\[[SsLlI.]\])*[\w\n\[\], -]*?([0-9]{4}|[0-9]{3,4} ?[?+])'
pubinfo = re.compile(pubstring, re.DOTALL)

nocitystring = r'\[[Ss]\.[IiLl]\.\]'
nocitypat = re.compile(nocitystring)

nopublisherstring = r'\[[Ss]\.[Nn]\.\]'
nopublisherpat = re.compile(nopublisherstring)

pagestring = r'[0-9]*[ \t]p\.'
pagepat = re.compile(pagestring)

commabracketstr = r'[,\[\]]'
combrackpat = re.compile(commabracketstr)

In [14]:
print(len(loc87monos_water))
loc87monos=re.sub(watermark, '', loc87monos_water)
print(len(loc87monos))

298376
289982


In [15]:
loc87monoslist = [listing.strip('\n') for listing in re.split(numberdiv, loc87monos)]

In [27]:
#loc87monoslist[:40]
for x in loc87monoslist[:40]:
    print(x)
    print(start.search(x))
    
    print(pubinfo.search(x))
    print(pubinfo.search(x).group())
    print(pubinfo.search(x).groups()[0])
    print(nocitypat.search(x))
    print(nopublisherpat.search(x))
        
    print(lccnpat.search(x))
    print(lcclass.search(x))
    print(pagepat.search(x))
    
    print('***\n\n')

An Cương. Tiềng trông Mê Linh; bìa và minh họa
của Tham Đức Tụ. In lần thứ 2. Hà Nội, Kim
Đổng, 1978.

PZ90.V5A5 1978 Orien Viet

80-984061

94	p. illus.
<_sre.SRE_Match object; span=(0, 9), match='An Cương.'>
<_sre.SRE_Match object; span=(77, 100), match=' Hà Nội, Kim\nĐổng, 1978'>
 Hà Nội, Kim
Đổng, 1978
1978
None
None
<_sre.SRE_Match object; span=(130, 139), match='80-984061'>
<_sre.SRE_Match object; span=(103, 128), match='PZ90.V5A5 1978 Orien Viet'>
<_sre.SRE_Match object; span=(141, 146), match='94\tp.'>
***


An Quân. Theo bác đi chiên dịch: kỷ niệm của những
người chiBn sĩ cánh vệ của bác. [s.l.] Văn Học Giải
Phóng, 1976.

MLCS83/8000 (P)

83-113774

223	p.

Ảnh, I-Si-Ca-Oa Bun-Dô;
see Ishikawa, Bun’ yõ
<_sre.SRE_Match object; span=(0, 8), match='An Quân.'>
<_sre.SRE_Match object; span=(88, 114), match='] Văn Học Giải\nPhóng, 1976'>
] Văn Học Giải
Phóng, 1976
1976
<_sre.SRE_Match object; span=(83, 89), match='[s.l.]'>
None
<_sre.SRE_Match object; span=(134, 143), match='83-1137

In [91]:
def parseRecord(inputstring):
    #come back to this after lunch: in what order to handle [sl], [sn] and splitting?
    
    if pubinfo.search(inputstring):
        publication = pubinfo.search(inputstring)
        try: 
            year = publication.groups()[0]
        except:
            year = None
        try:
            #publisher = publication.group().split(',')[1].strip('[] ')
            publisher = re.split(combrackpat, publication.group())[-2]
        except:
            publisher = None
        try:
            pubcity = re.split(combrackpat, publication.group().strip('[ '))[0]
            #added .strip() to get rid of leading bracket and avoid empty string
            #pubcity = publication.group().split(',')[0]
        except:
            pubcity = None
        
    else:
        print('No pub match')
        return None
    
    if nocitypat.search(inputstring):
        #if [s.l.] in record, set city to "none" (overrides previous) and "titleend" to its start
        pubcity = None
        publisher = publication.group().split(',')[0].strip('[] ')
        titleend = nocitypat.search(inputstring).start()
    else:
        titleend = publication.start()
    if nopublisherpat.search(inputstring):
        publisher = None
    
    
    if start.search(inputstring):
        firststring = start.search(inputstring).group()
        firststringend = start.search(inputstring).end()
        
        
        
        if 'and' in firststring:
            #this is an author; split appropriately
            if 'others' in firststring:
                authors = firststring[:firststring.index('and')].split(', ')
                
                print(1)
            else:
                authors = firststring.replace('and', ',').strip('. ').split(', ')
                print(2)
            print('Authors: ', authors)
            print('Title: ', inputstring[firststringend:titleend])
        else:
            #still figuring out
            if firststring.istitle():
                authors = firststring.strip('. '))
                print(3)
                print('Title: ', inputstring[firststringend:titleend])
            else:
                print('Title: ', firststring.strip('. '))
                print('no author')
                print(4)
    else:
    #'author' is empty, use all text up to "pubinfo" for title
        print('Title:', inputstring[:titleend])
    print('City: ', pubcity)
    print('Publisher: ', publisher)
    print('Year: ', year)

In [93]:
for x in loc87monoslist[80:120]:
    print(x)
    parseRecord(x)
    print('\n***\n')

Cách mạng khoa học kỹ thuật ỏ- trên thề giới và ỏ" Việt
Nam: thư mục chuyên để. Hà Nội, Bộ Văn Hóa và
Thông Tin, Thư Viện Quôc Gia Việt Nam, 1979.

Z7401.C13 1979 Orien Viet

82-102345

206	p.
Title: Cách mạng khoa học kỹ thuật ỏ- trên thề giới và ỏ" Việt
Nam: thư mục chuyên để.
City:  Hà Nội
Publisher:   Thư Viện Quôc Gia Việt Nam
Year:  1979

***

Cách mạng tháng Mười. In lần thứ 2 có bổ sung. Hà
Nội, Phổ Thông, 1977.

DK265.C28 1977 Orien Viet

82-184510

73 p. illus., plates.
Title:  Cách mạng tháng Mười
no author
4
City:  Hà
Nội
Publisher:   Phổ Thông
Year:  1977

***

Cách mạng tháng tám ( 1945). In lẩn thứ 4 có sửa chữa
và bổ sung. Hà Nội, Sự Thật, 1980.

DS556.8.C328 1980 Orien Viet

83-135617

195	p.
Title:  Cách mạng tháng tám ( 1945)
no author
4
City:  Hà Nội
Publisher:   Sự Thật
Year:  1980

***

Cách mạng và khoa học: nghiên cứu văn kiện đại hội
Đảngìãn thứ IV. Hà Nội, Khoa Học Xã Hội, 1977.

JQ898.D293C3 1977 Orien Viet

83-135047

295 p.

“ủy ban khoa học xã hội Việt Nam

In [10]:
for line in loc87monoslist:
    print()'Publisher: ', publisher.search(line))


SyntaxError: invalid syntax (<ipython-input-10-9c8e2db2e647>, line 2)